#Test Google Docs API

In [1]:
!pip install google-api-python-client google-auth google-auth-oauthlib google-auth-httplib2

In [2]:
from google.colab import auth
from googleapiclient.discovery import build

# Authenticate the user in Google Colab
auth.authenticate_user()

# Initialize the Google Docs API service
SCOPES = ['https://www.googleapis.com/auth/documents']
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
import google.auth

creds, _ = google.auth.default()
service = build('docs', 'v1', credentials=creds)

# Test API by creating a new Google Doc
document = service.documents().create(body={'title': 'Test Doc'}).execute()
print(f"Created document with ID: {document.get('documentId')}")

Created document with ID: 1AtudL2epBYsHfecS6ZsE_CHGlu6OARp58Vxm1I3YcaQ


#Ansible Health Assessment

In [4]:
import os
import re
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
import google.auth

# Authenticate and create the Google Docs API service
def authenticate_google_docs():
    from google.colab import auth
    auth.authenticate_user()

    try:
        # Authenticate using default credentials and create a service client
        creds, _ = google.auth.default()
        service = build('docs', 'v1', credentials=creds)
        return service
    except Exception as e:
        print(f"Error during authentication: {e}")
        raise

# Class for converting Markdown content to a Google Docs document
class MarkdownToGoogleDocs:
    def __init__(self, service, document_title):
        self.service = service  # Google Docs API service instance
        self.document_title = document_title  # Title of the document to be created
        try:
            self.document_id = self.create_google_doc()  # Create a Google Doc and get its ID
        except Exception as e:
            print(f"Error creating Google Doc: {e}")
            raise

    def create_google_doc(self):
        try:
            # Create a new Google Doc with the specified title and return its document ID
            doc = self.service.documents().create(body={"title": self.document_title}).execute()
            return doc['documentId']
        except Exception as e:
            print(f"Error during document creation: {e}")
            raise

    def append_content(self, requests):
        try:
            # Append content to the document using a batchUpdate request
            self.service.documents().batchUpdate(
                documentId=self.document_id, body={'requests': requests}
            ).execute()
        except Exception as e:
            print(f"Error appending content to document: {e}")
            raise

    def format_text(self, text, style, start_index):
        """Helper method to format text with a specified style (e.g., Heading 1)"""
        return {
            'updateParagraphStyle': {
                'range': {
                    'startIndex': start_index,
                    'endIndex': start_index + len(text) + 1  # Include newline
                },
                'paragraphStyle': {
                    'namedStyleType': style
                },
                'fields': 'namedStyleType'
            }
        }

    def format_footer(self, footer_text, start_index):
        """Helper method to format footer text with italic style and grey color"""
        return {
            'updateTextStyle': {
                'range': {
                    'startIndex': start_index,
                    'endIndex': start_index + len(footer_text)
                },
                'textStyle': {
                    'italic': True,
                    'foregroundColor': {
                        'color': {
                            'rgbColor': {'red': 0.5, 'green': 0.5, 'blue': 0.5}  # Grey color
                        }
                    }
                },
                'fields': 'italic,foregroundColor'
            }
        }

    def convert_markdown_to_google_doc(self, markdown_text):
        try:
            # Convert Markdown-formatted text into Google Docs-compatible requests
            lines = markdown_text.split('\n')
            requests = []

            # Ensure the document starts with an empty paragraph
            requests.append({
                'insertText': {
                    'location': {'index': 1},
                    'text': '\n'
                }
            })

            start_index = 1  # Tracks the current position in the document
            footer_lines = []  # Collect footer lines for separate formatting
            is_footer = False

            for line in lines:
                if line.strip() == '---':
                    # Start footer section when encountering '---'
                    is_footer = True
                    continue

                if is_footer:
                    footer_lines.append(line)
                    continue

                if line.startswith('# '):
                    # Main title - format as Heading 1
                    title = line.replace('# ', '')
                    requests.append({
                        'insertText': {
                            'location': {'index': start_index},
                            'text': title + '\n'
                        }
                    })
                    requests.append(self.format_text(title, 'HEADING_1', start_index))
                    start_index += len(title) + 1

                elif line.startswith('## '):
                    # Section headers - format as Heading 2
                    section_header = line.replace('## ', '')
                    requests.append({
                        'insertText': {
                            'location': {'index': start_index},
                            'text': section_header + '\n'
                        }
                    })
                    requests.append(self.format_text(section_header, 'HEADING_2', start_index))
                    start_index += len(section_header) + 1

                elif line.startswith('### '):
                    # Sub-section headers - format as Heading 3
                    subsection_header = line.replace('### ', '')
                    requests.append({
                        'insertText': {
                            'location': {'index': start_index},
                            'text': subsection_header + '\n'
                        }
                    })
                    requests.append(self.format_text(subsection_header, 'HEADING_3', start_index))
                    start_index += len(subsection_header) + 1

                elif line.startswith('- [ ]') or line.startswith('- [x]'):
                    # Interactive checkboxes
                    checkbox_text = line[6:].strip()
                    requests.append({
                        'insertText': {
                            'location': {'index': start_index},
                            'text': checkbox_text + '\n'
                        }
                    })
                    requests.append({
                        'createParagraphBullets': {
                            'range': {
                                'startIndex': start_index,
                                'endIndex': start_index + len(checkbox_text) + 1
                            },
                            'bulletPreset': 'BULLET_CHECKBOX'
                        }
                    })
                    for word in checkbox_text.split():
                        if word.startswith('@'):
                            # Highlight mentions (e.g., @username) in bold
                            mention_start = start_index + checkbox_text.find(word)
                            mention_end = mention_start + len(word)
                            requests.append({
                                'updateTextStyle': {
                                    'range': {
                                        'startIndex': mention_start,
                                        'endIndex': mention_end
                                    },
                                    'textStyle': {
                                        'bold': True
                                    },
                                    'fields': 'bold'
                                }
                            })
                    start_index += len(checkbox_text) + 1

                elif line.startswith('* ') or line.startswith('  * '):
                    # Nested bullet points
                    bullet_text = line.strip()
                    indentation = ' ' * (line.count('  ') * 2)  # Add spaces for indentation
                    requests.append({
                        'insertText': {
                            'location': {'index': start_index},
                            'text': indentation + bullet_text + '\n'
                        }
                    })
                    start_index += len(bullet_text) + len(indentation) + 1

                elif '@' in line:
                    # Handle mentions with bold formatting
                    requests.append({
                        'insertText': {
                            'location': {'index': start_index},
                            'text': line + '\n'
                        }
                    })
                    for word in line.split():
                        if word.startswith('@'):
                            mention_start = start_index + line.find(word)
                            mention_end = mention_start + len(word)
                            requests.append({
                                'updateTextStyle': {
                                    'range': {
                                        'startIndex': mention_start,
                                        'endIndex': mention_end
                                    },
                                    'textStyle': {
                                        'bold': True
                                    },
                                    'fields': 'bold'
                                }
                            })
                    start_index += len(line) + 1

                elif line.strip() == '':
                    # Empty lines
                    requests.append({
                        'insertText': {
                            'location': {'index': start_index},
                            'text': '\n'
                        }
                    })
                    start_index += 1

                else:
                    # Regular text
                    requests.append({
                        'insertText': {
                            'location': {'index': start_index},
                            'text': line + '\n'
                        }
                    })
                    start_index += len(line) + 1

            # Add footer at the end
            if footer_lines:
                footer_text = '\n'.join(footer_lines)
                requests.append({
                    'insertText': {
                        'location': {'index': start_index},
                        'text': footer_text + '\n'
                    }
                })
                requests.append(self.format_footer(footer_text, start_index))
                start_index += len(footer_text) + 1

            # Send the batchUpdate request
            self.append_content(requests)
        except Exception as e:
            print(f"Error during Markdown conversion: {e}")
            raise

# Main function to convert the provided Markdown content to a Google Doc
def main():
    markdown_notes = """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
    """

    try:
        service = authenticate_google_docs()
        converter = MarkdownToGoogleDocs(service, "Product Team Sync - May 15, 2023")
        converter.convert_markdown_to_google_doc(markdown_notes)
    except Exception as e:
        print(f"Error in main function: {e}")

if __name__ == "__main__":
    main()
